In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

from Faster_RCNN.Test import Test_Faster_RCNN

Couldn't import fast version of function compute_overlap, will use slow one. Check cython intallation


In [2]:
all_img_name= os.listdir('Data/test_img')

test_dataset= []
for name in tqdm(all_img_name):
    data= {}
    data['image_path']= 'Data/test_img/'+name
    test_dataset.append(data)
    
test_dataset

100%|██████████████████████████████████████████████████████████████████████████████████████████| 56/56 [00:00<?, ?it/s]


[{'image_path': 'Data/test_img/image_00000.jpg'},
 {'image_path': 'Data/test_img/image_00001.jpg'},
 {'image_path': 'Data/test_img/image_00002.jpg'},
 {'image_path': 'Data/test_img/image_00003.jpg'},
 {'image_path': 'Data/test_img/image_00004.jpg'},
 {'image_path': 'Data/test_img/image_00005.jpg'},
 {'image_path': 'Data/test_img/image_00006.jpg'},
 {'image_path': 'Data/test_img/image_00007.jpg'},
 {'image_path': 'Data/test_img/image_00008.jpg'},
 {'image_path': 'Data/test_img/image_00009.jpg'},
 {'image_path': 'Data/test_img/image_00010.jpg'},
 {'image_path': 'Data/test_img/image_00011.jpg'},
 {'image_path': 'Data/test_img/image_00012.jpg'},
 {'image_path': 'Data/test_img/image_00013.jpg'},
 {'image_path': 'Data/test_img/image_00014.jpg'},
 {'image_path': 'Data/test_img/image_00015.jpg'},
 {'image_path': 'Data/test_img/image_00016.jpg'},
 {'image_path': 'Data/test_img/image_00017.jpg'},
 {'image_path': 'Data/test_img/image_00018.jpg'},
 {'image_path': 'Data/test_img/image_00019.jpg'},


# Detection

In [3]:
import torch
import time
import pandas as pd
from torchvision.models.detection.transform import GeneralizedRCNNTransform

base_img_size= 1792
Test_CFG= {
    # [1792-128, 1792-64, 1792, 1792+64, 1792+128]
    'img_size': [base_img_size-192,
                 #base_img_size-128,
                 base_img_size-64,
                 base_img_size,
                 base_img_size+64,
                 #base_img_size+128,
                 base_img_size+192],
    'confidence': 0.001,
    'NMS_threshold': 0.6,
    'TTA': True,
    
    'load_model': 'test_cv_model/resnet152_1792_best/faster_rcnn_379.pth',
    'show_img': False,
    'show_score': False,
    'show_classes': False,
    'save_img_path': False, #'test/predict', # False則不儲存結果圖片
    'device': 'gpu:1',
    'class_mapping': False,
}



result= Test_Faster_RCNN(
            dataset= test_dataset[:],
            CFG= Test_CFG,
        )

using cuda:1


100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:53<00:00,  1.04it/s]


checking dataset


100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:50<00:00,  1.11it/s]


checking dataset


...

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [01:00<00:00,  1.08s/it]


checking dataset


100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [01:06<00:00,  1.19s/it]


checking dataset


100%|███████████████████████████████████████████████████████████████████████████████| 56/56 [00:00<00:00, 10991.67it/s]


In [4]:
submit= pd.DataFrame()

keep_classes= [1,2,3,4]
for pred in result:
    name= pred['image_path'].split('/')[-1]
    bbox= pred['bbox']
    label= pred['label'].astype(np.int)
    score= pred['score']
    for i in range(len(bbox)):
        if label[i] not in keep_classes: continue
        r= []
        bbox[i][2]-= bbox[i][0]
        bbox[i][3]-= bbox[i][1]
        r+= [name]
        r+= [label[i]]
        r+= bbox[i].astype(np.int).tolist()
        r+= [score[i]]
        submit= submit.append([r], ignore_index= True)


submit.columns= ['image_filename', 
              'label_id',
              'x',
              'y',
              'w',
              'h',
              'confidence']
submit.to_csv('submission.csv', index= False)
submit

,image_filename,label_id,x,y,w,h,confidence
0,image_00000.jpg,1,1682,3297,755,188,0.876293
1,image_00000.jpg,1,2448,3314,806,169,0.796912
2,image_00000.jpg,1,733,2605,125,151,0.788651
3,image_00000.jpg,1,705,2480,193,273,0.779468
4,image_00000.jpg,1,3153,225,1124,248,0.719317
...,...,...,...,...,...,...,...
23281,image_00055.jpg,1,1700,127,12,11,0.002567
23282,image_00055.jpg,4,1494,1465,8,8,0.002535
23283,image_00055.jpg,4,2595,130,45,42,0.002531
23284,image_00055.jpg,4,2158,1523,22,26,0.002526
